<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [211]:
from scipy.sparse import lil_matrix
import edward as ed
from edward.models import Beta
import tensorflow as tf
import numpy as np
import math

In [132]:
with open('kronecker-core-periphery-n1024-h10-r0_01-0_25-1000-cascades.txt','r') as f:
    
    # Store number of nodes
    numNodes = -1
    while True:
        if f.readline() == "\n":
            break
        numNodes+=1

    # Collect cascades into list
    v = []
    for line in f.readlines():
        v.append([float(l) for l in line.rstrip('\n').split(",")])

In [297]:
np_cascades = np.zeros((len(v),numNodes))
for row, cascade in enumerate(v):  
    c_nodes = [int(cascade[i*2]) for i in range(len(cascade)//2)]
    c_times = [cascade[i*2+1] for i in range(len(cascade)//2)]

    for col in range(len(c_nodes)):
        np_cascades[row][c_nodes[col]]=c_times[col]

dtype('float64')

In [89]:
def infectedCascade(cascade, start,N, T=10):
    A_inf = lil_matrix((cascade.shape[0],cascade.shape[0]))

    infected = np.nonzero(cascade)[0]
    c_nodes= cascade.argsort()[-len(infected):]

    for i in range(len(c_nodes)):
        for j in range(i):
            if cascade[j] < T:
                A_inf[(c_nodes[i],c_nodes[j])]=cascade[i]-cascade[j]
    
    return A_inf

In [88]:
def uninfectedCascade(cascade,start,N,T=10):
    nodes = {s for s in range(N)}
    A_uninf = lil_matrix((N,N))

    infected = np.nonzero(cascade)[0]
    c_nodes= cascade.argsort()[-len(infected):]

    for i in range(len(c_nodes)):
        for j in (nodes-set(c_nodes)):
            A_uninf[c_nodes[i],j]=T-cascade[i]

    return A_uninf

In [208]:
def sigmoid(x):
    try: 
        return 1 / (1 + math.exp(-x))
    except OverflowError:
        if x > 0: 
            return 1
        else: 
            return 0


In [272]:
numNodes = 5
#np_cascades.shape


In [276]:
A = Beta(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))
cascade_ph = Cascade(A)

sess.run(cascade_ph._log_prob(np.array([0,1,1,0,0])))

# dir(cascade_ph)

-36.875782253086534

In [285]:
np_cascades.shape

(1001, 1023)

In [277]:
data = {cascade_ph: np_cascades[0]}
inference = ed.MAP([A], data)

NameError: name 'np_cascades' is not defined

/Users/brianallen131/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


In [289]:
sess = ed.get_session()
tf.global_variables_initializer().run()

inference.run(n_iter=2500)

TypeError: __init__() got an unexpected keyword argument 'dtype'

In [314]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.framework import tensor_shape

from edward.models.random_variable import RandomVariable
from tensorflow.contrib.distributions import Distribution

try:
    from tensorflow.contrib.distributions import FULLY_REPARAMETERIZED
except Exception as e:
    raise ImportError("{0}. Your TensorFlow version is not supported.".format(e))

class Cascade(RandomVariable, Distribution):
    def __init__(self,
                params,
                seed_set = None,
                seed_set_param = 0.01,
                validate_args=False,
                allow_nan_stats=True,
                name="Cascade"):
        """Initialize a `Cascade` random variable.
        Args:
            params: tf.Tensor.
            Collection of samples. Its outer (left-most) dimension
            determines the number of samples.
        """
        parameters = locals()
        with tf.name_scope(name, values=[params]):
            with tf.control_dependencies([]):
                self._params = tf.identity(params, name="alpha")
                self._seed_set = seed_set
                self._seed_set_param = seed_set_param
                try:
                    self._n = tf.shape(self._params)[0] 
                except ValueError:  # scalar params
                    self._n = tf.constant(1)
                    
        super(Cascade, self).__init__(
            dtype=self._params.dtype,
            reparameterization_type=FULLY_REPARAMETERIZED,
            validate_args=validate_args,
            allow_nan_stats=allow_nan_stats,
            parameters=parameters,
            graph_parents=[self._params, self._n],
            name=name)
    
    @staticmethod
    def _param_shapes(sample_shape):
        return {"params": tf.convert_to_tensor(sample_shape, dtype=tf.int32)}

    @property
    def params(self):
        """Distribution parameter."""
        return self._params

    @property
    def n(self):
        """Number of samples."""
        return self._n

    def _batch_shape_tensor(self):
        return array_ops.shape(self.params)

    def _batch_shape(self):
        return self.params.shape[1:]

    def _event_shape_tensor(self):
        return tf.shape(self.params)[1:]

    def _event_shape(self):
        return tensor_shape.scalar()


    
    def _log_prob(self, value):
        # TODO
        alpha_tensor = tf.convert_to_tensor(tf.cast(self.params,dtype=tf.float64),dtype=tf.float64)
        
        I = tf.convert_to_tensor(infectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)
        U = tf.convert_to_tensor(uninfectedCascade(value,0,N=alpha_tensor.shape[0]).todense(),dtype = tf.float64)


        psi_1 = -tf.reduce_sum(tf.multiply(alpha_tensor,I))
        psi_2 = -tf.reduce_sum(tf.multiply(tf.transpose(alpha_tensor),U))
        
        zero_pos = tf.add(tf.multiply(tf.sign(I),-1),tf.ones((self.params.shape[0],self.params.shape[0]),dtype=tf.float64))
        psi_3 = tf.reduce_sum(tf.log(tf.add(zero_pos,tf.multiply(tf.transpose(alpha_tensor),tf.sign(I)))))

        return psi_1 + psi_2 + psi_3

    def _sample_n(self, n=1, seed=None):
        # generate the cascades
        #if self._seed_set: 
        #    seedset = self._seed_set
        #else: 
        #    seedset = np.random.binomial(1, self._seed_set_param,size=[N,1])
        
        #I_t = tf.convert_to_tensor(seedset)
        #I_t_1 = tf.zeros((N,1))
        A = self.parameters['params']
        N = A.shape[0]
        
        #seedset = np.random.randint(2, size=N)
        seedset = np.random.binomial(1, 0.0067,size=[N,1])
        data = np.zeros((n,N))
        for s in range(n):
            I_t = np.array(seedset).reshape(N,1)
            I_t_1 = np.zeros((N,1))
            #beta_tau = np.matmul(beta, tau)
            i = 1
            data1 = np.zeros((N,1))
            while not(np.array_equal(I_t,I_t_1)): 
                I_new = I_t - I_t_1
                data1 += I_new*i
                I_uninf = np.ones((N,1)) - I_t 
                AI = np.matmul(A.T, I_new)
                # calculate probability
                p = AI
                p = np.apply_along_axis(sigmoid,1, p).reshape(N,1) * I_uninf

                # draw random vector
                I_t_1 = I_t
                I_t = I_t_1 + np.random.binomial(1, p)
                i += 1
            data[s] = data1.flatten()
            
        
        return tf.convert_to_tensor(data.reshape((n,N)))
        """
                

        
        input_tensor = self.params
        if len(input_tensor.shape) == 0:
            input_tensor = tf.expand_dims(input_tensor, 0)
            multiples = tf.concat([tf.expand_dims(n, 0), [1] * len(self.event_shape)], 0)
            return tf.tile(input_tensor, multiples)
        else: 
            probs = tf.ones([self.n]) / tf.cast(self.n, dtype=tf.float32)
            cat = tf.contrib.distributions.Categorical(probs)
            indices = cat._sample_n(n, seed)
            print(input_tensor.shape)
            print(indices.shape)
            tensor = tf.gather(input_tensor, indices)
            print(tensor.shape)
            return tensor
        """

In [256]:
alpha_test = np.random.uniform(0,1,[5,5])
# print(v[0])
x = Cascade(alpha_test)
sess = tf.InteractiveSession()
sess.run(x.sample(100))

array([[2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 2.],
       ...,
       [2., 2., 2., ..., 2., 2., 2.],
       [2., 2., 2., ..., 2., 2., 3.],
       [2., 2., 2., ..., 2., 2., 2.]])

In [287]:
N = 100
D = 10

X = tf.placeholder(tf.float32, [N, D])
w = Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

qw = Normal(loc=tf.Variable,
            scale=tf.nn.softplus(tf.get_variable("qw/scale3", [D])))
qb = Normal(loc=tf.get_variable("qb/loc3", [1]),
            scale=tf.nn.softplus(tf.get_variable("qb/scale3", [1])))

In [289]:
X_train = np.random.binomial(1, 0.01,size=[N,D])
y_train = np.random.binomial(1, 0.01,size=[N])

inference = ed.KLqp({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(n_samples=5, n_iter=250)

/Users/jan-felix/Documents/Studium/MSc Data Science (Columbia)/ML Probabilistic Programming/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/jan-felix/Documents/Studium/MSc Data Science (Columbia)/ML Probabilistic Programming/src/edward/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


250/250 [100%] ██████████████████████████████ Elapsed: 2s | Loss: 94.919  6/250 [  2%]                                ETA: 88s | Loss: 126.223 

In [328]:
Aq = tf.Variable(tf.float32, [numNodes, numNodes])

TypeError: Failed to convert object of type <class 'tensorflow.python.framework.dtypes.DType'> to Tensor. Contents: <dtype: 'float32'>. Consider casting elements to a supported type.

In [323]:
from edward.models import PointMass
A = Beta(tf.ones([numNodes,numNodes]),tf.ones([numNodes,numNodes]))
cascade = Cascade(params=A)


AttributeError: 'Beta' object has no attribute 'T'

In [337]:
qA = PointMass(params= tf.get_variable("qA2", [numNodes,numNodes], dtype=tf.float32))

In [334]:
qA = tf.get_variable("qA1", [numNodes,numNodes])

In [335]:
qA

<tf.Variable 'qA1:0' shape=(80, 80) dtype=float32_ref>

In [339]:
X_train = np.random.binomial(1, 0.01,size=[N,numNodes])
inference = ed.MAP({A: qA}, data={cascade: X_train[0]})


/Users/jan-felix/Documents/Studium/MSc Data Science (Columbia)/ML Probabilistic Programming/src/edward/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/jan-felix/Documents/Studium/MSc Data Science (Columbia)/ML Probabilistic Programming/src/edward/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


In [279]:
numNodes = 5
data = np.random.binomial(1, 0.01,size=[100,numNodes])


inference = ed.MAP([A], data = {cascade: data})

TypeError: Key-value pair in data does not have same shape: (5,), (100, 5)

In [223]:
s = np.array([[1,2],[1,1]])
s2 = np.array([0,0])

In [252]:
x.shape

TensorShape([Dimension(100)])

In [225]:
s

array([[0, 0],
       [1, 1]])

In [184]:
alpha_test = np.ones((3,3),dtype=np.float64)/2

testcascade = Cascade(alpha_test)

sess = tf.InteractiveSession()
test_list = [1., 0., 0., 1.5]
sess.run(testcascade._log_prob(test_list))

ValueError: Cannot reshape a tensor with 5 elements to shape [1] (1 elements) for 'Cascade_66/sample/Reshape' (op: 'Reshape') with input shapes: [5,1], [1] and with input tensors computed as partial shapes: input[1] = [1].

In [237]:
sess.run(cascade_ph._log_prob(v[0]))

-24725.090304108082

In [183]:
from edward.models import Dirichlet, Beta, Multinomial, Bernoulli
K=100
N=50

gamma = Dirichlet(concentration=tf.ones([K]))
Pi = Beta(concentration0=tf.ones([K, K]), concentration1=tf.ones([K, K]))
Z = Multinomial(total_count=1.0, probs=gamma, sample_shape=N)
X = Bernoulli(probs=tf.matmul(Z, tf.matmul(Pi, tf.transpose(Z))))

gamma.event_shape

TensorShape([Dimension(100)])

In [114]:
x.parameters

{'allow_nan_stats': True,
 'loc': (0.0, 1.0, 6.0),
 'name': 'Normal_20/',
 'scale': (0.01, 0.01, 0.01),
 'validate_args': False}

In [119]:
x = ed.models.Normal(loc=(0.0,1.0,6.0), scale=(.01,.01,.01))
sess = tf.InteractiveSession()
sess.run()

TypeError: run() missing 1 required positional argument: 'fetches'

In [72]:
x._log_prob(1)

<tf.Tensor 'sub_26:0' shape=(3,) dtype=float32>

In [83]:
alpha_test = np.random.uniform(0,1,[5,5])

In [105]:
alpha_test

array([[0.70810882, 0.69454899, 0.63581996, 0.04190473, 0.51930298],
       [0.55419182, 0.66537161, 0.59641107, 0.98422131, 0.77345272],
       [0.28455863, 0.67817888, 0.28607481, 0.5308818 , 0.98624358],
       [0.91694389, 0.26283193, 0.99754276, 0.19616535, 0.32979132],
       [0.44641251, 0.59923134, 0.20658496, 0.70895587, 0.70404919]])

In [144]:
c = Cascade(alpha_test, seed_set=np.array([[1,0,0,1,0],[1,0,0,0,0]]))
sess = tf.InteractiveSession()
sess.run(c._n)

5

In [147]:
c.parameters

{'allow_nan_stats': True,
 'name': 'Cascade',
 'params': array([[0.70810882, 0.69454899, 0.63581996, 0.04190473, 0.51930298],
        [0.55419182, 0.66537161, 0.59641107, 0.98422131, 0.77345272],
        [0.28455863, 0.67817888, 0.28607481, 0.5308818 , 0.98624358],
        [0.91694389, 0.26283193, 0.99754276, 0.19616535, 0.32979132],
        [0.44641251, 0.59923134, 0.20658496, 0.70895587, 0.70404919]]),
 'seed_set': array([[1, 0, 0, 1, 0],
        [1, 0, 0, 0, 0]]),
 'seed_set_param': 0.01,
 'validate_args': False}

<tf.Tensor 'Cascade_21/strided_slice:0' shape=() dtype=int32>

In [ ]:
# build a custom variable with shape (4 Gamma distributions who are independent)